In [1]:
#!pip install -q openai

In [2]:
import os

import openai
import tiktoken

import achan_config as CFG
import achan_data.intents as INTENT                # определение интента запроса
import achan_data.common as COMMON                 # начальная настройка
import achan_data.delivery as DELIVERY             # "условия доставки"
import achan_data.delivery_alk as DELIVERY_ALK     # "доставка 18+"
import achan_data.delivery_faq as DELIVERY_FAQ     # "вопросы доставки"
import achan_data.delivery_free as DELIVERY_FREE   # "бесплатная доставка"
import achan_data.info as INFO                     # "об ашане"
import achan_data.refund_exceptions as REFUND_EXC  # "возврат исключения"
import achan_data.refund_food as REFUND_FOOD       # "возврат продовольственного"
import achan_data.refund_nonfood as REFUND_NONFOOD # "возврат непродовольственного"
import achan_data.shops as SHOPS                   # "адреса ашан"

In [3]:
print(CFG.SYSTEM)

 Вы полезный, отзывчивый, креативный, умный и очень дружелюбный помощник магазина Ашан. Вы помогаете покупателям отвечая на их вопросы.

Ваше имя Ашанчик.

Инструкции:
- Вы отвечаете только на вопросы, связанные с магазином Ашан. На посторонние вопросы отвечать строго запрещено.
- Если вы не уверены в ответе, вы можете сказать "Я не знаю" или "Я не уверен" и предложите пользователю переключить его на сотрудника магазина.



In [4]:
#model = "gpt-3.5-turbo"  # 4096 токенов
#model = "gpt-3.5-turbo-16k"  # 16384 токена
model = "gpt-4"   # 8192 токена
openai.api_key = CFG.CHAT_GPT_API_KEY
max_tokens =  1000
temperature = 0.1
token_limit = 8000

In [5]:
# настройка роли асистента для определения темы сообщения
intent_prompt=[
    {"role": "system", "content": INTENT.SYSTEM},
    {"role": "assistant", "content": INTENT.ASSISTANT}
]

# настраиваем роли и даем базу для ответов
messages=[
    # системная роль, чтобы задать поведение помошника
    {"role": "system", "content": CFG.SYSTEM},
    # промт для выяснения темы проблемы клиента
    {"role": "assistant", "content": COMMON.ASSISTANT}
]

In [6]:
# список доступных моделей
#models = openai.Model.list()
#for model in models.data:
#    print(model.id)

In [7]:
# функция подсчета числа токенов
def num_tokens_from_messages(messages, model=model):
    encoding= tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # каждое сообщение следует за <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # если есть имя, то роль опускается
                num_tokens += -1  # роль всегда обязательна и всегда 1 токен
    num_tokens += 2  # в каждом ответе используется <im_start> помощник
    return num_tokens

In [8]:
# функция делает запрос и возвращает ответ модели
def get_response(model="gpt-4", msg="", tokens=100, temp=0.1):
    # формируем запрос к модели
    completion = openai.ChatCompletion.create(
        model=model,
        messages=msg,
        max_tokens = tokens,
        temperature = temp
    )
    # получаем ответ
    chat_response = completion.choices[0].message.content
    return chat_response

In [9]:
content = ''
# цикл диалога
while not 'пока' in content.lower():
    # ввод пользователя
    content = input("Пользователь: ").strip()
    if content == "":
        content = "Привет! Как тебя зовут?"
    # добавляем сообщение пользователя
    messages.append({"role": "user", "content": content})
    intent_prompt.append({"role": "user", "content": content})
    
    # пытаемся получить тему сообщения
    intent = get_response(model=model, msg=intent_prompt, tokens=100, temp=0.1)
    # определяем совапдения по темам и загружаем нужный промт
    if "возврат продовольственного" in intent:
        messages[1] = {"role": "assistant", "content": REFUND_FOOD.ASSISTANT}
    elif "возврат непродовольственного" in intent:
        messages[1] = {"role": "assistant", "content": REFUND_NONFOOD.ASSISTANT}
    elif "возврат исключения" in intent:
        messages[1] = {"role": "assistant", "content": REFUND_EXC.ASSISTANT}
    elif "условия доставки" in intent:
        messages[1] = {"role": "assistant", "content": DELIVERY.ASSISTANT}
    elif "бесплатная доставка" in intent:
        messages[1] = {"role": "assistant", "content": DELIVERY_FREE.ASSISTANT}
    elif "вопросы доставки" in intent:
        messages[1] = {"role": "assistant", "content": DELIVERY_FAQ.ASSISTANT}
    elif "доставка 18+" in intent:
        messages[1] = {"role": "assistant", "content": DELIVERY_ALK.ASSISTANT}
    elif "адреса ашан" in intent:
        messages[1] = {"role": "assistant", "content": SHOPS.ASSISTANT}
    elif "об ашане" in intent:
        messages[1] = {"role": "assistant", "content": INFO.ASSISTANT}
    else:
        messages[1] = {"role": "assistant", "content": COMMON.ASSISTANT}
    
    # общее число токенов
    conv_history_tokens = num_tokens_from_messages(messages)
    print(f"Токенов: {conv_history_tokens}, интент: {intent}")
    # удаляем прошлые сообщения, если число токенов превышает лимиты
    while conv_history_tokens + max_tokens >= token_limit:
        del messages[2]
        conv_history_tokens = num_tokens_from_messages(messages)

    # формируем запрос и получаем ответ
    chat_response = get_response(model=model, msg=messages, tokens=max_tokens, temp=temperature)

    # выводим ответ
    print(f'''
Ашанчик: {chat_response}
''')
    
    # сохраняем контекст диалога
    messages.append({"role": "assistant", "content": chat_response})

Пользователь: Добрый день!
Токенов: 352, интент: "другая тема"

Ашанчик: Добрый день! Чем я могу помочь вам сегодня?

Пользователь: купил у вас битые яйца
Токенов: 668, интент: возврат продовольственного

Ашанчик: Очень жаль, что вы столкнулись с такой проблемой. Если вы обнаружили, что яйца повреждены, вам следует обратиться в службу поддержки клиентов Ашан. Пожалуйста, предоставьте фотографии поврежденного товара и чека. Это поможет ускорить обработку вашего обращения.

Вы можете связаться с нами через:

- Форму обратной связи на нашем сайте (https://www.auchan.ru/feedback/)
- Контактный центр по телефону 8-800-700-5-800 (звонок бесплатный)
- Чат-бот в Viber (viber://pa?chatURI=auchanrusbot)

Приносим извинения за доставленные неудобства.

Пользователь: хорошо, спасибо, так и сделаю
Токенов: 659, интент: "другая тема"

Ашанчик: Вы всегда можете обратиться за помощью! Надеюсь, ваша проблема будет решена как можно скорее. Если у вас возникнут еще вопросы, не стесняйтесь задавать. Удачн

In [10]:
messages

[{'role': 'system',
  'content': ' Вы полезный, отзывчивый, креативный, умный и очень дружелюбный помощник магазина Ашан. Вы помогаете покупателям отвечая на их вопросы.\n\nВаше имя Ашанчик.\n\nИнструкции:\n- Вы отвечаете только на вопросы, связанные с магазином Ашан. На посторонние вопросы отвечать строго запрещено.\n- Если вы не уверены в ответе, вы можете сказать "Я не знаю" или "Я не уверен" и предложите пользователю переключить его на сотрудника магазина.\n'},
 {'role': 'assistant',
  'content': '\n Уточни намерение клиента, какая у него проблема:\n \n- Если клиент хочет узнать о возврате некачественного, сломанного или бракованного товара, спроси его какой конкретно товар он хочет вернуть.\n- Если клиент спрашивает о доставке, спроси в каком городе он находится.\n- В остальных случаях спроси что клиент хочет узнать о магазине Ашан.\n'},
 {'role': 'user', 'content': 'Добрый день!'},
 {'role': 'assistant',
  'content': 'Добрый день! Чем я могу помочь вам сегодня?'},
 {'role': 'user